In [2]:
import warnings                                  # `do not disturbe` mode
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [45]:
df = pd.read_csv('FAOSTAT_downloaded20210119.csv')
foods = pd.read_csv('foods.csv')

In [46]:
#Bind old (-2013) and new time series (2014-): Food Balances and Producer prices
df.Domain.replace('Food Balances (old methodology and population)', 'FoodBalances', inplace=True)
df.Domain.replace('New Food Balances', 'FoodBalances', inplace=True)
df.Domain.replace('Producer Prices', 'ProducerPrices', inplace=True)
df.Domain.replace('Producer Prices (old series)', 'ProducerPrices', inplace=True)
df.Domain.replace('Food Supply - Crops Primary Equivalent', 'FoodSupply', inplace=True)
df.Domain.replace('Crops', 'CropProduction', inplace=True)
df.Domain.replace('Value of Agricultural Production', 'ValueAgProduction', inplace=True)
df.Element.replace('Food supply quantity (kg/capita/yr)', 'FoodPerCapita', inplace=True)
df.Element.replace('Producer Price Index (2014-2016 = 100)', 'PriceIndex', inplace=True)
df.Element.replace('Gross Production Value (constant 2014-2016 million US$)','GrossProductionValue', inplace=True)
df.Element.replace('Area harvested','AreaHarvested', inplace=True)

#merge with dictionary including "Food groups". Category 2 based on HDDS.
#if you want to filter out non-food, processed foods and beverages, select 'Crop' and 'CropProducts'
df = df.merge(foods, how='left', on='Item')

#select only 'crop' and 'cropProducts'
df = df[((df.Category1 == 'Crop') | (df.Category1 == 'CropProducts'))]
df = df[['Domain', 'Area', 'Element', 'Year', 'Unit', 'Value', 'name']]
df.rename(columns={'name':'Item'}, inplace=True)

#Calculate for LVB
#whyprice index does not work?
LVB = df.groupby(['Domain', 'Year', 'Element', 'Item']).mean()
LVB['Area'] = 'LVB'
LVB.reset_index(inplace=True)
df = pd.concat([LVB, df])

#change unit
df.Unit[(df.Element== 'Production')] = '1000 tones'
df.Unit[(df.Element== 'Food')] = '1000 tones'
df.Unit[(df.Element== 'FoodPerCapita')] = 'kg/capita/year'
df.Unit[(df.Element== 'PriceIndex')] = 'PriceIndex'
df.Unit[(df.Element== 'GrossProductionValue')] = 'million US$'
df.Unit[(df.Element== 'AreaHarvested')] = '1000 ha'

df.Element[(df.Domain == 'CropProduction') & (df.Element == 'Production')] = 'CropProduction'


In [ ]:
df[(df.Year == 2018) & (df.Item == 'Maize') & (df.Element == 'Food') & (df.Area == 'LVB')]

In [47]:
# Food Balance
df1 = df[(df.Domain == 'FoodBalances') &
         (df.Year >= 1991) & (df.Year <= 2019)]
a = df1[(df1.Element == 'Production') ].groupby(['Item', 'Area']).mean().reset_index()
a = a.rename(columns={'Value':'Production'})
b = df1[ (df1.Element == 'Food')].groupby(['Item', 'Area']).mean().reset_index()
b = b.rename(columns={'Value':'Food'})
d = df1[(df1.Element == 'Import Quantity')].groupby(['Item', 'Area']).mean().reset_index()
d = d.rename(columns={'Value':'Import'})
e = df1[(df1.Element == 'Export Quantity')].groupby(['Item', 'Area']).mean().reset_index()
e = e.rename(columns={'Value':'Export'})
f = df1[(df1.Element == 'Domestic supply quantity')].groupby(['Item', 'Area']).mean().reset_index()
f = f.rename(columns={'Value':'DomesticSupply'})
g = df1[(df1.Element == 'FoodPerCapita')].groupby(['Item', 'Area']).mean().reset_index()
g = g.rename(columns={'Value':'FoodPerCapita'})

#Producer Price Index
df3 = df[(df.Domain == 'ProducerPrices') &
         (df.Year >= 1991) & (df.Year <= 2019)] 
h = df3[(df3.Element == 'PriceIndex') ].groupby(['Item', 'Area']).mean().reset_index()
h = h.rename(columns = {'Value':'PriceIndex'})

#ValueAgProduction
df4 = df[(df.Domain == 'ValueAgProduction') & 
         (df.Year >= 1991) & (df.Year <= 2019)] 
i = df4[(df4.Element == 'GrossProductionValue') ].groupby(['Item', 'Area']).mean().reset_index()
i = i.rename(columns = {'Value':'GrossProductionValue'})

#CropProduction 
df5 = df[(df.Domain == 'CropProduction') &
         (df.Year >= 1991) & (df.Year <= 2019)] 
j = df5[(df5.Element == 'AreaHarvested') ].groupby(['Item', 'Area']).mean().reset_index()
j = j.rename(columns = {'Value':'AreaHarvested'})

#drop year
porVar = [a, b, g, h, i, j]
for cadaUno in porVar:
    cadaUno.drop('Year', axis=1, inplace=True)

#merge
df2 = a.merge(b, how='outer', on=['Item', 'Area'])
for cadaUno in porVar[2:]:
    df2 = df2.merge(cadaUno, how='outer', on=['Item', 'Area'])
    
#round
df2['FoodSelfSufficiency'] = df2.Production / df2.Food  *100
df2['AreaHarvested'] = (df2['AreaHarvested'] / 1000)
porInd = ['Production', 'Food', 'FoodPerCapita', 'PriceIndex', 
          'GrossProductionValue', 'AreaHarvested', 'FoodSelfSufficiency']
for cadaUno in porInd:
    df2[cadaUno] = df2[cadaUno].round(1)


In [6]:
def potentialCompetitor (variable, country, number):
    a = df2[df2['Area'] == country].sort_values(variable, ascending=False).head(number)
    a.reset_index(inplace=True)
    a.drop('index', axis=1, inplace=True)         
    a.index += 1
    a.rename(columns={'Item':'TopCrops'}, inplace=True)

    return a

In [7]:
def graphCompetitors (variable, country, number):
    datos = df.rename(columns={'Item' : 'TopCrops'})
    datos = datos.sort_values('Year', ascending=True)
        
    topCrops = potentialCompetitor(variable, country, number)
    datos = datos.merge(topCrops['TopCrops'], how='right', on='TopCrops')
    
    datitos = datos[(datos.Element == variable)]
    
    fig = px.line(datitos,
                  x='Year', 
                  y = 'Value',
                  color='TopCrops', 
                  facet_col='Area',
                  facet_col_wrap=2,
                  title=variable, 
                  labels = {'Value': datitos.Unit.iloc[1]})

    fig.show()

In [58]:
potentialCompetitor('AreaHarvested', 'UGA', 10)

,TopCrops,Area,Production,Food,FoodPerCapita,PriceIndex,GrossProductionValue,AreaHarvested,FoodSelfSufficiency
1,Plantains and others,UGA,NaN,NaN,NaN,NaN,NaN,1299.4,NaN
2,Maize,UGA,1618.4,996.0,32.6,NaN,NaN,801.3,162.5
3,Cassava,UGA,3549.0,2410.0,85.1,NaN,NaN,559.4,147.3
4,Sweet potatoes,UGA,2124.7,1909.5,69.4,NaN,NaN,500.0,111.3
5,Sorghum,UGA,384.3,124.1,4.5,NaN,NaN,319.5,309.6
6,"Coffee, green",UGA,NaN,NaN,NaN,NaN,NaN,317.7,NaN
7,Millet,UGA,465.1,337.3,13.3,NaN,NaN,306.6,137.9
8,"Groundnuts, with shell",UGA,NaN,NaN,NaN,NaN,NaN,278.0,NaN
9,Sesame seed,UGA,105.4,48.8,1.8,NaN,NaN,200.3,216.0
10,Sunflower seed,UGA,151.8,34.3,1.0,NaN,NaN,151.1,442.7


In [59]:
graphCompetitors('AreaHarvested', 'UGA', 10)